
# DATABASE SETUP
# WROKING!!!!
![](https://i.imgur.com/nnZkSj8.png)
![](https://i.imgur.com/fSqozNf.png)


# Load dataset

In [ ]:
%run ../../src/start.py

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

# load data

In [ ]:
# helper function
def bucket_raw_path(bucket_name,path_dir):
    ''''''
    raw_path = f's3://{bucket_name}/{path_dir}'
    return raw_path

In [ ]:
raw = bucket_raw_path('dend-data',f'capstone/load-data/')
raw

In [ ]:
file = wr.s3.list_objects(raw)[0]
file

In [ ]:
df = wr.s3.read_csv(file)

In [ ]:
df.info()

In [ ]:
df.head()

# TEST IMPORT

# Police Beat

## insert record to policebeat table

# Load all police beats

In [ ]:
df.head()

## OFFENSE_TABLE

In [ ]:
offense_list = df.offense_type.unique().tolist()
offense_list

In [ ]:
df_offense = pd.DataFrame(offense_list,columns=['offense_name'])
df_offense

In [ ]:
df_offense = df_offense.reset_index()
df_offense.rename(columns={'index': 'offense_pk'}, inplace=True)
df_offense.offense_pk = df_offense.offense_pk +1
df_offense

## BEAT_TABLE

In [ ]:
beat_list = df.beat.unique().tolist()
df_beat = pd.DataFrame(beat_list,columns=['beat_name'])

df_beat = df_beat.reset_index()
df_beat.rename(columns={'index': 'beat_pk'}, inplace=True)
df_beat.beat_pk = df_beat.beat_pk +1
df_beat.info()

In [ ]:
df_beat.head()

In [ ]:
df.head()

## PREMISE_DESCRIPTION_TABLE

In [ ]:
premise_list = df.premise_description.unique().tolist()
df_premise = pd.DataFrame(premise_list,columns=['premise_name'])

df_premise = df_premise.reset_index()
df_premise.rename(columns={'index': 'premise_pk'}, inplace=True)
df_premise.premise_pk = df_premise.premise_pk +1
df_premise.info()

In [ ]:
df_premise.head()

In [ ]:
df_premise.head()

## Address_TABLE

In [ ]:
df.head()

In [ ]:
df.block_range.unique()

In [ ]:
df.block_range = df.block_range.str.strip()

In [ ]:
df.block_range.replace('UNK','10-100',inplace=True)
df.block_range.replace('1.1103e+006-1.1104e+006','10-100',inplace=True)

In [ ]:
df.block_range.unique()

In [ ]:
def block_range_split(df):
    '''split blockrange col values
    then give median value as a string'''
    first = df.block_range.str.split(pat='-',expand=True)[0].astype('int')
    second = df.block_range.str.split(pat='-',expand=True)[1].astype('int')
    med = np.ceil((second + first)/2).astype('int')
    med = med.astype('str')
    street = df.street_name
    res = med + " " + street
    return res

In [ ]:
df['full_address'] = block_range_split(df)

In [ ]:
df.head()

## FULL ADDRESS

In [ ]:
add_list = df.full_address.unique().tolist()
df_address = pd.DataFrame(add_list,columns=['full_address'])

df_address = df_address.reset_index()
df_address.rename(columns={'index': 'address_pk'}, inplace=True)
df_address.address_pk = df_address.address_pk +1
df_address.info()

In [ ]:
df_address.head()

# DATE_TIME_TABLE

In [ ]:
df.head()

In [ ]:
time_list = df.date_time.unique().tolist()

df_time = pd.DataFrame(time_list,columns=['date_time'])


df_time = df_time.reset_index()
df_time.rename(columns={'index': 'date_time_pk'}, inplace=True)
df_time.date_time_pk = df_time.date_time_pk +1
df_time.info()

In [ ]:
df_time.head()

In [ ]:
# convert timestamp column to datetime
t = pd.to_datetime(df_time['date_time'])
# insert time data records
time_data = (df_time.date_time_pk,df_time.date_time,t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year,t.dt.weekday)
column_labels = ('date_time_pk','date_time', 'hour', 'day','week','month','year','weekday')
date_time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
date_time_df.head()

In [ ]:
date_time_df.info()

# Crime_Fact_Table

In [ ]:
df.head()

In [ ]:
df = df[['date_time',
 'offenses',
 'offense_type',
 'beat',
 'premise_description',
 'temp',
 'feels_like',
 'humidity_per',
 'rain_vol_1h_mm',
 'snow_vol_1h_mm',
 'full_address']]

In [ ]:
df.columns.tolist()

In [ ]:
crime_fact = df.copy()
crime_fact.head()

In [ ]:
crime_fact.head()

# PREMISE_SPLIT

In [ ]:
crime_fact = crime_fact.merge(df_premise, left_on='premise_description', right_on='premise_name', suffixes=(True,True))
crime_fact.drop(['premise_description', 'premise_name'], axis=1,inplace=True)

In [ ]:
crime_fact.head()

# ADDRESS_SPLIT

In [ ]:
crime_fact = crime_fact.merge(df_address, left_on='full_address', right_on='full_address', suffixes=(True,True))
crime_fact.drop(['full_address'], axis=1,inplace=True)

In [ ]:
crime_fact.head()

# BEAT SPLIT

In [ ]:
crime_fact =crime_fact.merge(df_beat, left_on='beat', right_on='beat_name', suffixes=(True,True))
crime_fact.drop(['beat_name','beat'], axis=1,inplace=True)

In [ ]:
crime_fact.head()

# OFFENSE_SPLIT

In [ ]:
crime_fact = crime_fact.merge(df_offense, left_on='offense_type', right_on='offense_name', suffixes=(True,True))
crime_fact.drop(['offense_type','offense_name'], axis=1,inplace=True)

In [ ]:
crime_fact.head()

# TIME SPLIT

In [ ]:
crime_fact.head()

In [ ]:
crime_fact = crime_fact.merge(date_time_df, left_on='date_time', right_on='date_time', suffixes=(True,True))
crime_fact.drop(['date_time','hour','day','week','month','year','weekday'], axis=1,inplace=True)

In [ ]:
crime_fact.head()

In [ ]:
crime_fact.head()

In [ ]:
crime_fact.rename(columns={'offenses': 'numoffenses', 'rain_vol_1h_mm': 'rain','snow_vol_1h_mm':'snow','humidity_per':'humidity'}, inplace=True)

In [ ]:
crime_fact.head()

In [ ]:
crime_fact = crime_fact.reset_index()
crime_fact.rename(columns={'index': 'pk'}, inplace=True)
crime_fact.pk = crime_fact.pk +1
crime_fact.info()

In [ ]:
crime_fact.head()

In [ ]:
crime_fact.columns.tolist()

In [ ]:
crime_fact = crime_fact[[ 'pk', 'numoffenses','temp', 'humidity', 'feels_like',
 'rain',
 'snow',
'date_time_pk',
 'address_pk',
 'premise_pk',
'offense_pk',
 'beat_pk']]

In [ ]:
crime_fact.head()

# UPLOAD DTA

In [ ]:
# load secret keys
db_host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')
db_port = os.environ.get('DB_PORT')


conn = psycopg2.connect(database=db_name,
user=db_user,
password=db_pass,
host=db_host,
port=db_port)
conn.set_session(autocommit=True)
cur = conn.cursor()


In [ ]:
df_premise.head()

# premise upload

In [ ]:
for i in df_premise.values.tolist():
    cur.execute(premise_table_insert, i)
    conn.commit()

# Address upload

In [ ]:
df_address.head()

In [ ]:
for i in df_address.values.tolist():
    cur.execute(address_table_insert, i)
    conn.commit()

# df_beat

In [ ]:
df_beat.head()

In [ ]:
for i in df_beat.values.tolist():
    cur.execute(police_beat_table_insert, i)
    conn.commit()

# df_offense load

In [ ]:
df_offense

In [ ]:
for i in df_offense.values.tolist():
    cur.execute(offense_table_insert, i)
    conn.commit()

## date_time_df

In [ ]:
date_time_df.head()

In [ ]:
for i in date_time_df.values.tolist():
    cur.execute(datetime_table_insert, i)
    conn.commit()

In [ ]:
datetime_table_insert

# crime fact upload

In [ ]:
crime_fact.head()

In [ ]:
for i in crime_fact.values.tolist():
    cur.execute(crime_fact_table_insert, i)
    conn.commit()